# Loading Ameriflux stations to database
Logan Gall

29 May 2024

In [3]:
import rpy2
import os
os.environ['R_HOME'] = 'C:\Program Files\R\R-4.3.2'
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr, data
import psycopg2
import csv
utils = importr('utils')
base = importr('base')
amr = importr('amerifluxr')

In [4]:
dbname = 'postgres'
user = 'postgres'
password = 'Passwordd'
host = 'localhost'
station_data_path = './amf_station_data.csv'

In [5]:
sites = amr.amf_site_info()
with (robjects.default_converter + pandas2ri.converter).context():
  pd_from_r_df = robjects.conversion.get_conversion().rpy2py(sites)

pd_from_r_df
pd_from_r_df.to_csv(station_data_path, index = False)

In [6]:
# Table creation SQL statement (adjust according to your data)
# Example assumes a table with columns: id (integer), name (text), and age (integer)
create_table_sql = """
CREATE TABLE IF NOT EXISTS amf_stations (
    site_id VARCHAR(255) PRIMARY KEY,
    site_name TEXT,
    country TEXT,
    state TEXT,
    url_ameriflux TEXT,
    location_lat VARCHAR(255),
    location_long VARCHAR(255),
    location_elev VARCHAR(255),
    data_policy TEXT,
    data_start NUMERIC,
    data_end NUMERIC
);
"""

# Connect to your postgres DB
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute the SQL to create the table if it doesn't exist
cur.execute(create_table_sql)
conn.commit()  # Commit immediately after creating the table

# Close the cursor and connection
cur.close()
conn.close()

In [7]:
# Connect to your postgres DB
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

# Open a cursor to perform database operations
cur = conn.cursor()

with open(station_data_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    count = 0
    for row in reader:
        count += 1
        if (count % 10000) == 0:
            conn.commit()
            print('commit')
        cur.execute("""
            INSERT INTO amf_stations(site_id, site_name, country, state, url_ameriflux, location_lat, location_long, location_elev, data_policy, data_start, data_end)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (site_id) DO NOTHING;
        """, (row['SITE_ID'], row['SITE_NAME'], row['COUNTRY'], row['STATE'], row['URL_AMERIFLUX'], row['LOCATION_LAT'], row['LOCATION_LONG'], row['LOCATION_ELEV'], row['DATA_POLICY'], row['DATA_START'], row['DATA_END']))

conn.commit()

# Close the cursor and connection
cur.close()
conn.close()